In [ ]:
# загрузим библиотеки и установим опции

from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVR
from sklearn.linear_model import Ridge
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as st
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_df = pd.read_csv('data/train.csv', index_col='ID')
test_df = pd.read_csv('data/test.csv', index_col='ID')

In [ ]:
def encode_categorial(x):
    x = x.reset_index()
    le = LabelEncoder()
    col = x.columns.difference(['ID'])
    print(pd.Series(le.fit_transform(x[col])))
    x[col] = pd.Series(le.fit_transform(x[col])).astype('category', ordered=False)
    x = x.set_index('ID')
    return x

train_idx = train_df.index.values
test_idx = test_df.index.values

y_train = train_df['y']
full_df = pd.concat([train_df.drop('y', axis=1), test_df])

categorial_fields = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

full_df_1 = full_df[full_df.columns.difference(categorial_fields)].apply(lambda x: x.astype('category', ordered=False), axis=0)
#train_df_3 = train_df[categorial_fields].apply(lambda x: encode_categorial(x), axis=0)
full_df_2 = pd.get_dummies(full_df[categorial_fields])
full_df_2 = full_df_2.apply(lambda x: x.astype('category', ordered=False), axis=0)

full_df = pd.concat([full_df_1, full_df_2], axis=1)

X_train = full_df.loc[train_idx]
X_test = full_df.loc[test_idx]

In [ ]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted, out_file, target='y', index=None, index_label="ID"):
    if index is None:
        index = np.arange(1, predicted.shape[0] + 1)
    predicted_df = pd.DataFrame(predicted,
                                index = index,
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
full_df_xgb = full_df[full_df.columns].apply(lambda x: x.astype('bool'), axis=0)
X_train_xgb = full_df_xgb.loc[train_idx]
X_test_xgb = full_df_xgb.loc[test_idx]

In [ ]:
params = {  
    "n_estimators": [100],
    "max_depth": [3,],
    "learning_rate": [0.05],
}

xgbreg = XGBRegressor() 
grid = GridSearchCV(xgbreg, params, n_jobs=-1, cv=5, scoring='r2')
grid.fit(X_train_xgb, y_train)
grid.best_score_

In [ ]:
y_test = grid.best_estimator_.predict(X_test_xgb)
write_to_submission_file(y_test, 'submit_xbg_lr_005_md3_nest100_.csv', index=X_test.index.values)